**Load the dataset**

In [ ]:
!kaggle datasets download -d haithemhermessi/sanad-dataset

Dataset URL: https://www.kaggle.com/datasets/haithemhermessi/sanad-dataset
License(s): U.S. Government Works
 96% 63.0M/65.7M [00:04<00:00, 21.4MB/s]
100% 65.7M/65.7M [00:04<00:00, 16.0MB/s]


In [ ]:
import zipfile

# Unzip the dataset
with zipfile.ZipFile('sanad-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset_directory')


In [ ]:
!pip install langchain langchain langchain_community langchain-google-genai langchain_experimental sentence-transformers langchain_chroma langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.5/150.5 kB 16.1 MB/s

In [ ]:
import os
from langchain.schema import Document

In [ ]:
def read_and_create_documents(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return Document(page_content=text, metadata={"source": file_path})

# Iterate through directories and files
def process_directory(directory):
    documents = []
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if os.path.isdir(category_path):
            for filename in os.listdir(category_path):
                file_path = os.path.join(category_path, filename)
                if file_path.endswith('.txt'):
                    documents.append(read_and_create_documents(file_path))
    return documents

In [ ]:
dataset_directory = '/content/dataset_directory'
all_documents = process_directory(dataset_directory)

len(all_documents)

45500

In [ ]:
all_documents[0]

Document(metadata={'source': '/content/dataset_directory/Culture/0213.txt'}, page_content='استضاف بيت الشعر مساء أمس الأول الشاعر يوسف أبولوز في أمسية حضرها حشد من جمهور الشعر والإعلاميين. قدم الأمسية الزميل حسام ميرو بمداخلة نقدية عن تجربة يوسف أبولوز الشعرية أوضح فيها فرادة صوت أبولوز في المشهد الشعري الفلسطيني حيث لم يقع في أفخاخ الايديولوجيا التي غالباً ما تقضي على الشعر وجمالياته لمصلحة الشعار العام. ومن يقرأ أعماله منذ صباح الكاتيوشا أيها المخيم حتى مجموعته الأخيرة خط الهزلاج يدرك أن هذا الشاعر مسكون بتراث حي ومتجدد هو تراث الطبيعة.ومن قصيدة قافية مكررة من ديوان خط الهزلاج قرأ أبولوز مقاطع تعبر عن مواجهة مفتوحة وعلنية بين الذات الطامحة إلى الحرية في مطلقها وواقع طافح بالمرارة والألم حيث قال:وأنا،أكتب السيرة.. الله يرحمني لا الأناس،لقد صغت حرفين من أصل حرفولونت ثيابي بهذا الدم الحلو.. دم الغزال،الذي شمني ثم طاف،وبعد.. ارتجل:الضحى للغزالات،ثم دماءك فيهاهنا في العروق كلام وماء،سماء من الأرجوان،وطين له لذعة الحب، ثمة امرأةبلغت سن يأسينيأس المرايا.. ويأسي.وفي مجموعة من القصائد القصيرة

**Encode the data into vectore database**

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from tqdm import tqdm

persist_directory = "/content/drive/MyDrive/Electro_Pi_Capston_2/vectorstore/"

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

vectorstore = Chroma.from_documents(documents=all_documents[:41666], embedding=embedding_model,
                                    persist_directory=persist_directory)

In [ ]:
# Save the vectorstore in drive
vectorstore.persist()

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

**LLMs Model Setup**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

google_api_key = userdata.get('google_api_key')
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",max_tokens=500,google_api_key = google_api_key)

**Prompt Setup**

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
"أنت مساعد لمهام الإجابة على الأسئلة."
" استخدم الأجزاء التالية من السياق المسترجع للإجابة علي السؤال في ايجاز "
"\n\n"
"{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "هل استضاف بيت الشعر مساء أمس الأول الشاعر يوسف أبولوز ؟"})
print(response["answer"])

نعم، استضاف بيت الشعر مساء أمس الأول الشاعر يوسف أبولوز. 



**Load the Vector database from drive**

In [ ]:
!pip install gdown

In [ ]:
import gdown

file_id = "1-PE7eJwdjrUVYx7YFAtwE7g1frYvKAKc"
downloaded_file_path = "/content/chroma.sqlite3"

# Construct the Google Drive download URL
gdrive_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file using gdown
gdown.download(gdrive_url, downloaded_file_path, quiet=False)

# Ensure the file was downloaded correctly
if os.path.exists(downloaded_file_path):
    print(f"File downloaded successfully: {downloaded_file_path}")
else:
    print("File download failed.")


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

persist_directory = "/content"

# Initialize the embeddings model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Load the Chroma vectorstore using the persisted SQLite file
vectorstore = Chroma(embedding_function=embedding_model, persist_directory=persist_directory)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

**Making UI using Gradio**

In [ ]:
import gradio as gr
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define the function to process the input and return the outputs
def qa_system(question):
    response = rag_chain.invoke({"input": question})
    retrieved_docs = retriever.invoke(question)
    retrieved_content = "\n\n".join([doc.page_content for doc in retrieved_docs])
    return response["answer"], retrieved_content

# Create the Gradio interface with 1 input and 2 outputs
interface = gr.Interface(
    fn=qa_system,
    inputs="text",
    outputs=[
        gr.Textbox(label="Answer to the Question"),
        gr.Textbox(label="Most Similar Documents"),
    ],
    title="Question-Answering System",
    description="Enter a question to get relevant articles and the best answer."
)

interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://62e2899e9b88b3654f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
